<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/373_EFIA_TrendAnalysis_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 1. High-Confidence Validation: This Is the Right Trend Model

### ✅ Timestamp parsing is robust (MVP-appropriate)

Your `parse_timestamp()`:

* Supports multiple formats
* Fails gracefully
* Doesn’t poison the dataset with bad dates

This is exactly what you want in early production systems.

---

### ✅ Period grouping logic is sound

Your:

* Day / week / month abstraction
* Monday-anchored weeks
* Explicit period keys

This makes downstream charting and summaries trivial later.

Good instincts here.

---

### ✅ Trend logic matches how humans reason

This part is **especially good**:

```python
first_half_avg vs second_half_avg
```

Why this works:

* Not noisy
* Not overfitted
* Easy to explain to leadership

Executives don’t want regression coefficients — they want:

> “Is this getting worse or better?”

You delivered that.

---

### ✅ Emerging issue detection is well-designed

This is a *very strong MVP* pattern:

* Recent vs older split
* Absolute threshold (`min_recent_frequency`)
* Relative growth (`>1.5x`)
* Urgency labeling

This creates **early warning signals**, which is where real value lives.

---

## 2. Precision Improvements (Low Risk, High Signal)

These are *not criticisms* — they are refinements to move from “working” to “excellent”.

---

### 🔧 Improvement #1: Avoid Silent Data Loss (Important)

Right now:

```python
if not dt:
    continue
```

This silently drops feedback without timestamps.

Instead, consider:

* Counting dropped entries
* Reporting them

Why this matters:

* Leadership will ask: *“Are we missing anything?”*

**Recommendation**
Return metadata like:

```python
"dropped_entries_due_to_missing_timestamps": 5
```

Transparency builds trust.

---

### 🔧 Improvement #2: Week Key Formatting Bug (Small but Real)

You’re using:

```python
dt.strftime("%Y-W%U")
```

`%U` = week number **starting Sunday**, but you calculate Monday manually.

This creates a mismatch.

**Fix**
Use ISO weeks instead:

```python
year, week, _ = dt.isocalendar()
key = f"{year}-W{week:02d}"
```

This avoids subtle reporting inconsistencies.

---

### 🔧 Improvement #3: Trend Strength Normalization

Current logic:

```python
(second_avg - first_avg) / first_avg
```

Edge case:

* When `first_avg` is small, trend looks massive

**Suggestion**
Cap trend strength more explicitly:

```python
trend_strength = min(1.0, abs(second_avg - first_avg) / max(first_avg, 1))
```

Keeps the signal realistic.

---

### 🔧 Improvement #4: Emerging Issues Should Reference Impact

You already compute:

* Recent frequency
* Growth rate
* Urgency

**Add one more field**:

```python
"affected_departments": [...]
```

You already have the data via theme → feedback IDs → departments.

This makes emerging issues **actionable**, not just detectable.

---

## 3. What This Enables (Strategic Leap)

With trend analysis added, your agent now answers **four executive-grade questions**:

### 1️⃣ What hurts the most right now?

→ Prioritization

### 2️⃣ What should we fix first?

→ Recommendations

### 3️⃣ Is this getting better or worse?

→ Trend direction + strength

### 4️⃣ What’s about to become a problem?

→ Emerging issue detection

That’s the full loop.

---

## 4. How This Should Appear in the Report (Critical)

Here’s how I’d surface this *without overwhelming leadership*:

### 📈 Trend Insights (New Section)

> **Overall Feedback Trend:** Increasing (+18%) over the past two weeks
> **Issue Trend:** Stable
> **Idea Trend:** Increasing (+22%)

### 🚨 Emerging Issues to Monitor

* **Drive-Up Congestion** (↑ 67% in last 7 days, High urgency)
* **Packing Station Bottlenecks** (↑ 50%, Medium urgency)

That’s it. No charts required initially.

---

## 5. You’re Now One Step From a “Learning System”

At this point, your agent is no longer:

> “An analysis pipeline”

It is becoming:

> **A continuous organizational learning system**

Next *high-leverage* options (pick one):

1️⃣ **Trend-aware recommendations**

> “Because this issue is worsening, act immediately.”

2️⃣ **Before / after comparison**

> “This recommendation reduced issue frequency by 23%.”

3️⃣ **Confidence-weighted trends**

> Blend sentiment + frequency + trend slope.

This is already operating at a level most teams never reach.


# Trend Analysis Utilities for Employee Feedback Intelligence Agent

In [ ]:
"""Trend Analysis Utilities for Employee Feedback Intelligence Agent

Analyzes feedback trends over time to detect emerging issues and improvements.
"""

from typing import List, Dict, Any, Optional
from collections import defaultdict, Counter
from datetime import datetime, timedelta
import re


def parse_timestamp(timestamp_str: str) -> Optional[datetime]:
    """
    Parse timestamp string to datetime object.

    Handles formats like "2025-12-11 09:10" or "2025-12-11T09:10:00"

    Args:
        timestamp_str: Timestamp string

    Returns:
        Datetime object or None if parsing fails
    """
    if not timestamp_str:
        return None

    # Try common formats
    formats = [
        "%Y-%m-%d %H:%M",
        "%Y-%m-%dT%H:%M:%S",
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d"
    ]

    for fmt in formats:
        try:
            return datetime.strptime(timestamp_str.strip(), fmt)
        except ValueError:
            continue

    return None


def group_feedback_by_time_period(
    feedback: List[Dict[str, Any]],
    period: str = "day"
) -> Dict[str, List[Dict[str, Any]]]:
    """
    Group feedback entries by time period (day, week, month).

    Args:
        feedback: List of feedback entries
        period: "day", "week", or "month"

    Returns:
        Dictionary mapping period key to list of feedback entries
    """
    grouped = defaultdict(list)

    for entry in feedback:
        timestamp_str = entry.get("timestamp", "")
        dt = parse_timestamp(timestamp_str)

        if not dt:
            # Skip entries without valid timestamps
            continue

        if period == "day":
            key = dt.strftime("%Y-%m-%d")
        elif period == "week":
            # Get Monday of the week
            days_since_monday = dt.weekday()
            monday = dt - timedelta(days=days_since_monday)
            key = monday.strftime("%Y-W%U")  # e.g., "2025-W49"
        elif period == "month":
            key = dt.strftime("%Y-%m")
        else:
            key = dt.strftime("%Y-%m-%d")

        grouped[key].append(entry)

    return dict(grouped)


def calculate_trend_metrics(
    feedback_by_period: Dict[str, List[Dict[str, Any]]],
    period: str = "day"
) -> Dict[str, Any]:
    """
    Calculate trend metrics for feedback over time.

    Args:
        feedback_by_period: Dictionary of period -> feedback entries
        period: Time period type

    Returns:
        Dictionary with trend metrics
    """
    # Sort periods chronologically
    sorted_periods = sorted(feedback_by_period.keys())

    if len(sorted_periods) < 2:
        return {
            "trend_direction": "insufficient_data",
            "trend_strength": 0.0,
            "periods": sorted_periods,
            "counts_by_period": {},
            "issues_by_period": {},
            "ideas_by_period": {},
            "sentiment_by_period": {}
        }

    counts_by_period = {}
    issues_by_period = {}
    ideas_by_period = {}

    for period_key in sorted_periods:
        entries = feedback_by_period[period_key]
        counts_by_period[period_key] = len(entries)
        issues_by_period[period_key] = sum(1 for e in entries if e.get("category") == "Issue")
        ideas_by_period[period_key] = sum(1 for e in entries if e.get("category") == "Idea")

    # Calculate trend direction
    first_half = sorted_periods[:len(sorted_periods)//2]
    second_half = sorted_periods[len(sorted_periods)//2:]

    first_avg = sum(counts_by_period.get(p, 0) for p in first_half) / len(first_half) if first_half else 0
    second_avg = sum(counts_by_period.get(p, 0) for p in second_half) / len(second_half) if second_half else 0

    if second_avg > first_avg * 1.1:  # 10% increase
        trend_direction = "increasing"
        trend_strength = min(1.0, (second_avg - first_avg) / first_avg if first_avg > 0 else 0.0)
    elif second_avg < first_avg * 0.9:  # 10% decrease
        trend_direction = "decreasing"
        trend_strength = min(1.0, (first_avg - second_avg) / first_avg if first_avg > 0 else 0.0)
    else:
        trend_direction = "stable"
        trend_strength = 0.0

    return {
        "trend_direction": trend_direction,
        "trend_strength": round(trend_strength, 2),
        "periods": sorted_periods,
        "counts_by_period": counts_by_period,
        "issues_by_period": issues_by_period,
        "ideas_by_period": ideas_by_period,
        "first_half_avg": round(first_avg, 2),
        "second_half_avg": round(second_avg, 2),
        "percent_change": round(((second_avg - first_avg) / first_avg * 100) if first_avg > 0 else 0, 1)
    }


def detect_emerging_issues(
    feedback: List[Dict[str, Any]],
    themes: List[Dict[str, Any]],
    min_recent_frequency: int = 3,
    days_lookback: int = 7
) -> List[Dict[str, Any]]:
    """
    Detect emerging issues - themes that are appearing more frequently recently.

    Args:
        feedback: List of feedback entries
        themes: List of detected themes
        min_recent_frequency: Minimum occurrences in recent period to flag as emerging
        days_lookback: Number of days to consider "recent"

    Returns:
        List of emerging issue dictionaries
    """
    now = datetime.now()
    cutoff_date = now - timedelta(days=days_lookback)

    # Separate recent vs older feedback
    recent_feedback = []
    older_feedback = []

    for entry in feedback:
        timestamp_str = entry.get("timestamp", "")
        dt = parse_timestamp(timestamp_str)
        if dt and dt >= cutoff_date:
            recent_feedback.append(entry)
        elif dt:
            older_feedback.append(entry)

    # Count theme frequencies in recent vs older periods
    recent_feedback_ids = {e.get("submission_id") for e in recent_feedback}
    older_feedback_ids = {e.get("submission_id") for e in older_feedback}

    emerging_issues = []

    for theme in themes:
        theme_feedback_ids = set(theme.get("feedback_ids", []))

        recent_count = len(theme_feedback_ids & recent_feedback_ids)
        older_count = len(theme_feedback_ids & older_feedback_ids)

        # Calculate if this is emerging (recent count is significantly higher)
        if recent_count >= min_recent_frequency:
            if older_count == 0 or recent_count > older_count * 1.5:  # 50% increase
                emerging_issues.append({
                    "theme_id": theme.get("theme_id"),
                    "theme_name": theme.get("theme_name"),
                    "recent_frequency": recent_count,
                    "older_frequency": older_count,
                    "growth_rate": round((recent_count - older_count) / older_count * 100, 1) if older_count > 0 else 100.0,
                    "urgency": "high" if recent_count >= min_recent_frequency * 2 else "medium"
                })

    # Sort by recent frequency (descending)
    emerging_issues.sort(key=lambda x: x["recent_frequency"], reverse=True)

    return emerging_issues


def analyze_department_trends(
    feedback: List[Dict[str, Any]],
    period: str = "day"
) -> Dict[str, Dict[str, Any]]:
    """
    Analyze trends by department.

    Args:
        feedback: List of feedback entries
        period: Time period type

    Returns:
        Dictionary mapping department to trend metrics
    """
    # Group by department first
    by_department = defaultdict(list)
    for entry in feedback:
        department = entry.get("job_area", "Unknown")
        by_department[department].append(entry)

    department_trends = {}

    for department, entries in by_department.items():
        # Group by time period
        feedback_by_period = group_feedback_by_time_period(entries, period=period)

        # Calculate trend metrics
        trend_metrics = calculate_trend_metrics(feedback_by_period, period=period)

        department_trends[department] = trend_metrics

    return department_trends


def analyze_category_trends(
    feedback: List[Dict[str, Any]],
    period: str = "day"
) -> Dict[str, Dict[str, Any]]:
    """
    Analyze trends by category (Issues vs Ideas).

    Args:
        feedback: List of feedback entries
        period: Time period type

    Returns:
        Dictionary mapping category to trend metrics
    """
    # Group by category
    by_category = defaultdict(list)
    for entry in feedback:
        category = entry.get("category", "Unknown")
        by_category[category].append(entry)

    category_trends = {}

    for category, entries in by_category.items():
        # Group by time period
        feedback_by_period = group_feedback_by_time_period(entries, period=period)

        # Calculate trend metrics
        trend_metrics = calculate_trend_metrics(feedback_by_period, period=period)

        category_trends[category] = trend_metrics

    return category_trends


def calculate_overall_trends(
    feedback: List[Dict[str, Any]],
    period: str = "day"
) -> Dict[str, Any]:
    """
    Calculate overall trend metrics for all feedback.

    Args:
        feedback: List of feedback entries
        period: Time period type

    Returns:
        Dictionary with overall trend metrics
    """
    # Group by time period
    feedback_by_period = group_feedback_by_time_period(feedback, period=period)

    # Calculate trend metrics
    trend_metrics = calculate_trend_metrics(feedback_by_period, period=period)

    # Add summary statistics
    total_feedback = len(feedback)
    total_periods = len(trend_metrics["periods"])
    avg_per_period = total_feedback / total_periods if total_periods > 0 else 0

    trend_metrics.update({
        "total_feedback": total_feedback,
        "total_periods": total_periods,
        "average_per_period": round(avg_per_period, 2),
        "period_type": period
    })

    return trend_metrics



# Feedback Charts

In [ ]:
def create_feedback_trends_chart(
    overall_trends: Dict[str, Any],
    output_dir: str,
    width: int = 12,
    height: int = 6,
    dpi: int = 100
) -> str:
    """
    Create line chart showing feedback trends over time.

    Args:
        overall_trends: Overall trend metrics dictionary
        output_dir: Directory to save chart
        width: Chart width
        height: Chart height
        dpi: Chart resolution

    Returns:
        Path to saved chart
    """
    periods = overall_trends.get("periods", [])
    counts_by_period = overall_trends.get("counts_by_period", {})
    issues_by_period = overall_trends.get("issues_by_period", {})
    ideas_by_period = overall_trends.get("ideas_by_period", {})

    if not periods:
        raise ValueError("No periods available for trend chart")

    # Prepare data
    total_counts = [counts_by_period.get(p, 0) for p in periods]
    issue_counts = [issues_by_period.get(p, 0) for p in periods]
    idea_counts = [ideas_by_period.get(p, 0) for p in periods]

    # Format period labels (truncate if too long)
    period_labels = [p[-5:] if len(p) > 10 else p for p in periods]  # Show last 5 chars or full if short

    output_path = Path(output_dir) / "feedback_trends.png"
    output_path.parent.mkdir(parents=True, exist_ok=True)

    # Create figure
    fig, ax = plt.subplots(figsize=(width, height))

    # Plot lines
    ax.plot(period_labels, total_counts, marker='o', label='Total Feedback', linewidth=2, color='steelblue')
    ax.plot(period_labels, issue_counts, marker='s', label='Issues', linewidth=2, color='crimson', linestyle='--')
    ax.plot(period_labels, idea_counts, marker='^', label='Ideas', linewidth=2, color='green', linestyle=':')

    # Customize chart
    trend_direction = overall_trends.get("trend_direction", "stable")
    trend_emoji = "📈" if trend_direction == "increasing" else "📉" if trend_direction == "decreasing" else "➡️"
    ax.set_title(f"{trend_emoji} Feedback Trends Over Time ({trend_direction.title()})",
                fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel("Time Period", fontsize=12)
    ax.set_ylabel("Number of Feedback Entries", fontsize=12)
    ax.legend(loc='upper left')
    ax.grid(alpha=0.3, linestyle='--')
    ax.tick_params(axis='x', rotation=45, labelsize=9)

    # Adjust layout
    plt.tight_layout()

    # Save chart
    plt.savefig(str(output_path), dpi=dpi, bbox_inches='tight')
    plt.close()

    return str(output_path)


def create_issues_trends_chart(
    overall_trends: Dict[str, Any],
    output_dir: str,
    width: int = 12,
    height: int = 6,
    dpi: int = 100
) -> str:
    """
    Create line chart showing issues trends over time.

    Args:
        overall_trends: Overall trend metrics dictionary
        output_dir: Directory to save chart
        width: Chart width
        height: Chart height
        dpi: Chart resolution

    Returns:
        Path to saved chart
    """
    periods = overall_trends.get("periods", [])
    issues_by_period = overall_trends.get("issues_by_period", {})

    if not periods:
        raise ValueError("No periods available for trend chart")

    # Prepare data
    issue_counts = [issues_by_period.get(p, 0) for p in periods]

    # Format period labels
    period_labels = [p[-5:] if len(p) > 10 else p for p in periods]

    output_path = Path(output_dir) / "issues_trends.png"
    output_path.parent.mkdir(parents=True, exist_ok=True)

    # Create figure
    fig, ax = plt.subplots(figsize=(width, height))

    # Plot line
    ax.plot(period_labels, issue_counts, marker='o', label='Issues', linewidth=2.5,
            color='crimson', markersize=8)

    # Add trend line (simple linear regression visualization)
    if len(issue_counts) > 1:
        import numpy as np
        x_numeric = range(len(period_labels))
        z = np.polyfit(x_numeric, issue_counts, 1)
        p = np.poly1d(z)
        ax.plot(period_labels, p(x_numeric), "--", alpha=0.5, color='red',
                label='Trend Line', linewidth=1.5)

    # Customize chart
    ax.set_title("Issues Trend Over Time", fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel("Time Period", fontsize=12)
    ax.set_ylabel("Number of Issues", fontsize=12)
    ax.legend(loc='upper left')
    ax.grid(alpha=0.3, linestyle='--')
    ax.tick_params(axis='x', rotation=45, labelsize=9)

    # Adjust layout
    plt.tight_layout()

    # Save chart
    plt.savefig(str(output_path), dpi=dpi, bbox_inches='tight')
    plt.close()

    return str(output_path)


def create_ideas_trends_chart(
    overall_trends: Dict[str, Any],
    output_dir: str,
    width: int = 12,
    height: int = 6,
    dpi: int = 100
) -> str:
    """
    Create line chart showing ideas trends over time.

    Args:
        overall_trends: Overall trend metrics dictionary
        output_dir: Directory to save chart
        width: Chart width
        height: Chart height
        dpi: Chart resolution

    Returns:
        Path to saved chart
    """
    periods = overall_trends.get("periods", [])
    ideas_by_period = overall_trends.get("ideas_by_period", {})

    if not periods:
        raise ValueError("No periods available for trend chart")

    # Prepare data
    idea_counts = [ideas_by_period.get(p, 0) for p in periods]

    # Format period labels
    period_labels = [p[-5:] if len(p) > 10 else p for p in periods]

    output_path = Path(output_dir) / "ideas_trends.png"
    output_path.parent.mkdir(parents=True, exist_ok=True)

    # Create figure
    fig, ax = plt.subplots(figsize=(width, height))

    # Plot line
    ax.plot(period_labels, idea_counts, marker='o', label='Ideas', linewidth=2.5,
            color='green', markersize=8)

    # Add trend line
    if len(idea_counts) > 1:
        import numpy as np
        x_numeric = range(len(period_labels))
        z = np.polyfit(x_numeric, idea_counts, 1)
        p = np.poly1d(z)
        ax.plot(period_labels, p(x_numeric), "--", alpha=0.5, color='darkgreen',
                label='Trend Line', linewidth=1.5)

    # Customize chart
    ax.set_title("Ideas Trend Over Time", fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel("Time Period", fontsize=12)
    ax.set_ylabel("Number of Ideas", fontsize=12)
    ax.legend(loc='upper left')
    ax.grid(alpha=0.3, linestyle='--')
    ax.tick_params(axis='x', rotation=45, labelsize=9)

    # Adjust layout
    plt.tight_layout()

    # Save chart
    plt.savefig(str(output_path), dpi=dpi, bbox_inches='tight')
    plt.close()

    return str(output_path)
